<a href="https://colab.research.google.com/github/Rafiy27/Tugas-Akhir-Hate-Speech/blob/main/Corpus_Building_Feature_Extraction_(FastText)_Feature_Expansion_(TF_IDF)_Wikidump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Library

In [1]:
!pip install pandas openpyxl

In [2]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 845.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199774 sha256=df841431332d90c510500637dd5b83fe66dad73228e2905aaa1d501a57263fbb
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [3]:
!pip install requests

# Load & Preprocess Data Tweet

## Import Library & Load Dataset

In [4]:
import requests
import bz2
import xml.etree.ElementTree as ET
import fasttext
import time
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Download and extract data in chunks
url = 'https://dumps.wikimedia.org/idwiki/latest/idwiki-latest-pages-articles.xml.bz2'
file_path = 'idwiki-latest-pages-articles.xml.bz2'
chunk_size = 1024

response = requests.get(url, stream=True)
response.raise_for_status()

with open(file_path, 'wb') as file:
    for chunk in response.iter_content(chunk_size=chunk_size):
        file.write(chunk)

# Function to parse and process XML data in chunks
def parse_xml(file_path, num_articles=None):
    with bz2.open(file_path, 'rt', encoding='utf-8') as f:
        article = {}
        count = 0  # Counter to track the number of articles processed
        for line in f:
            line = line.strip()
            if line.startswith("<title>"):
                article["title"] = line[len("<title>"):-len("</title>")]
            elif line.startswith("<text "):
                text = ""
                for text_line in f:
                    if "</text>" in text_line:
                        text += text_line.split("</text>")[0]
                        break
                    else:
                        text += text_line
                article["text"] = text
                yield (article["title"], article["text"])
                article = {}

                count += 1  # Increment the counter
                if num_articles and count >= num_articles:
                    break  # Stop processing if the specified number of articles is reached

# Parse and process XML data in chunks
num_articles = 100  # 5% of the dataset
corpus_generator = parse_xml(file_path, num_articles=num_articles)
corpus = [next(corpus_generator) for _ in range(num_articles)]

In [5]:
import bz2
import xml.etree.ElementTree as ET

def count_articles(file_path):
    count = 0  # Initialize counter for articles

    with bz2.open(file_path, 'rt', encoding='utf-8') as f:
        inside_page = False  # Flag to track if currently inside a <page> tag

        for line in f:
            line = line.strip()

            if line == "<page>":
                inside_page = True  # Start of a new article
            elif line == "</page>":
                inside_page = False  # End of the current article
                count += 1  # Increment article counter

    return count

file_path = 'idwiki-latest-pages-articles.xml.bz2'
num_articles = count_articles(file_path)
print(f"Number of articles in the dataset: {num_articles}")

Number of articles in the dataset: 1648722


In [6]:
# Print the first three articles from the corpus
counter = 0
for title, text in corpus:
    print(f"Title: {title}")
    print(f"Text: {text[:500]}...")  # Print only the first 500 characters of the text for brevity
    print("-" * 50)  # Print a separator line between articles
    counter += 1
    if counter >= 3:
        break

Title: Asam deoksiribonukleat
Text: [[Berkas:DNA animation.gif|jmpl|Gambaran tiga dimensi DNA]]{{genetika}}
'''Asam deoksiribonukleat''', lebih dikenal dengan singkatan '''DNA''' ([[bahasa Inggris]]: '''''d'''eoxyribo'''n'''ucleic '''a'''cid''), adalah salah satu jenis [[asam nukleat]] yang memiliki kemampuan [[Hereditas|pewarisan]] sifat. Keberadaan asam deoksiribonukleat ditemukan di dalam [[nukleoprotein]] yang membentuk [[inti sel]]. [[James Dewey Watson]] dan [[Francis Crick]] merupakan ilmuwan pertama yang mengajukan model s...
--------------------------------------------------
Title: Asam Deoksiribosanukleat
Text:       <sha1>49bjs0f6ys3j5ymkyg8c41ino347ykz</sha1>
    </revision>
  </page>
  <page>
    <title>Anwar Sadat</title>
    <ns>0</ns>
    <id>3</id>
    <revision>
      <id>21063385</id>
      <parentid>19395623</parentid>
      <timestamp>2022-05-05T11:26:32Z</timestamp>
      <contributor>
        <username>InternetArchiveBot</username>
        <id>1223933</id>
      

In [7]:
# Save the corpus to a text file
output_file_path = "corpus_wikidump.txt"
with open(output_file_path, "w", encoding="utf-8") as file:
    for title, text in corpus:
        file.write(f"Title: {title}\n")
        file.write(f"Text: {text}\n\n")

# Train FastText Model

In [8]:
import fasttext
import time

# Train FastText model
model_path = 'trained_model_wikidump.bin'
print("Starting the FastText training process...")
start_time = time.time()
model = fasttext.train_unsupervised(output_file_path, model='skipgram')
model.save_model(model_path)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Model trained and saved successfully in {elapsed_time:.2f} seconds!")

Starting the FastText training process...
Model trained and saved successfully in 394.03 seconds!


In [9]:
# Load the trained FastText model
model = fasttext.load_model(model_path)

## Ekstraksi Fitur

In [10]:
import pandas as pd
import numpy as np
import fasttext

# Create a DataFrame from the parsed corpus
df = pd.DataFrame(corpus, columns=['title', 'text'])

# Function to extract feature vectors for a given text
def extract_features(text):
    words = text.split()
    feature_vectors = []
    for word in words:
        vector = model.get_word_vector(word)
        feature_vectors.append(vector)
    return feature_vectors

# Apply feature extraction to the 'text' column
df['feature_vectors'] = df['text'].apply(extract_features)

# Convert the list of feature vectors into a single array for each row
df['feature_vectors'] = df['feature_vectors'].apply(np.array)

# Save the DataFrame with feature vectors to a CSV file
df.to_csv('data_with_features.csv', index=False)

# Display the DataFrame with feature vectors
print(df[['title', 'text', 'feature_vectors']].head())

                      title  \
0    Asam deoksiribonukleat   
1  Asam Deoksiribosanukleat   
2              Azhar Mansor   
3                 Arkeologi   
4               Antropologi   

                                                text  \
0  [[Berkas:DNA animation.gif|jmpl|Gambaran tiga ...   
1        <sha1>49bjs0f6ys3j5ymkyg8c41ino347ykz</s...   
2  '''Datuk Azhar Mansor''' adalah orang [[Malays...   
3  {{Ilmu}}\n[[Berkas:Bulgandry Aboriginal Site.J...   
4  {{Zoologi}}\n'''Antropologi''' adalah ilmu ten...   

                                     feature_vectors  
0  [[-0.068555005, 0.40348157, 1.2137008, -0.1272...  
1  [[-0.07424517, -0.019402059, 0.08449326, 0.016...  
2  [[-0.38065776, 0.056810483, 0.49693507, 0.1475...  
3  [[-0.70683825, -0.15112978, 1.1264209, 0.36113...  
4  [[-0.6152643, -0.25781155, 0.6689638, 0.200405...  


In [15]:
df

,title,text,feature_vectors
0,Asam deoksiribonukleat,[[Berkas:DNA animation.gif|jmpl|Gambaran tiga ...,"[[-0.068555005, 0.40348157, 1.2137008, -0.1272..."
1,Asam Deoksiribosanukleat,<sha1>49bjs0f6ys3j5ymkyg8c41ino347ykz</s...,"[[-0.07424517, -0.019402059, 0.08449326, 0.016..."
2,Azhar Mansor,'''Datuk Azhar Mansor''' adalah orang [[Malays...,"[[-0.38065776, 0.056810483, 0.49693507, 0.1475..."
3,Arkeologi,{{Ilmu}}\n[[Berkas:Bulgandry Aboriginal Site.J...,"[[-0.70683825, -0.15112978, 1.1264209, 0.36113..."
4,Antropologi,{{Zoologi}}\n'''Antropologi''' adalah ilmu ten...,"[[-0.6152643, -0.25781155, 0.6689638, 0.200405..."
...,...,...,...
995,Sitoplasma,'''Sitoplasma''' adalah bagian [[sel (biologi)...,"[[-0.5021529, -0.23019293, 0.25113472, 0.34530..."
996,Soto bandung,<sha1>c0bp72uygo4t24toimesri4fd42tp98</s...,"[[-0.0610977, -0.01826957, 0.099067636, 0.0278..."
997,Aksara Kanji,<sha1>dr35x91ivwltxqxrz7ty22vnj81ee6t</s...,"[[-0.05445163, 0.008003685, 0.09989598, 0.0192..."
998,Karnaparwa,'''Karnaparwa''' merupakan kitab ke-8 dari ser...,"[[-0.46792656, -0.050959587, 0.27269533, 0.317..."


# Ekspansi Fitur (TF IDF)

In [12]:
# Initialize the TF-IDF vectorizer with max_features set to 5000
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialize the TF-IDF vectorizer with max_features set to 5000
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the 'text' to compute TF-IDF scores
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Combine the original DataFrame with the TF-IDF DataFrame
df_combined = pd.concat([df, tfidf_df], axis=1)

# Save the combined DataFrame to a CSV file
df_combined.to_csv('data_with_features_and_tfidf.csv', index=False)

# Display the combined DataFrame
print(df_combined[['title', 'text', 'feature_vectors'] + list(tfidf_vectorizer.get_feature_names_out())].head())

                      title     title  \
0    Asam deoksiribonukleat  0.050660   
1  Asam Deoksiribosanukleat  0.045511   
2              Azhar Mansor  0.000000   
3                 Arkeologi  0.000000   
4               Antropologi  0.090286   

                                                text      text  \
0  [[Berkas:DNA animation.gif|jmpl|Gambaran tiga ...  0.001983   
1        <sha1>49bjs0f6ys3j5ymkyg8c41ino347ykz</s...  0.016230   
2  '''Datuk Azhar Mansor''' adalah orang [[Malays...  0.000000   
3  {{Ilmu}}\n[[Berkas:Bulgandry Aboriginal Site.J...  0.000000   
4  {{Zoologi}}\n'''Antropologi''' adalah ilmu ten...  0.000000   

                                     feature_vectors        00       000  \
0  [[-0.068555005, 0.40348157, 1.2137008, -0.1272...  0.002812  0.000000   
1  [[-0.07424517, -0.019402059, 0.08449326, 0.016...  0.000000  0.000000   
2  [[-0.38065776, 0.056810483, 0.49693507, 0.1475...  0.000000  0.000000   
3  [[-0.70683825, -0.15112978, 1.1264209, 0.36113...

In [16]:
df_combined

,title,text,feature_vectors,00,000,001,01,02,03,04,...,yuri,yurisdiksi,yusuf,zakharia,zaman,zat,zefanya,zh,zodiak,zona
0,Asam deoksiribonukleat,[[Berkas:DNA animation.gif|jmpl|Gambaran tiga ...,"[[-0.068555005, 0.40348157, 1.2137008, -0.1272...",0.002812,0.000000,0.003495,0.00000,0.001898,0.005840,0.000000,...,0.0,0.004195,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,Asam Deoksiribosanukleat,<sha1>49bjs0f6ys3j5ymkyg8c41ino347ykz</s...,"[[-0.07424517, -0.019402059, 0.08449326, 0.016...",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,Azhar Mansor,'''Datuk Azhar Mansor''' adalah orang [[Malays...,"[[-0.38065776, 0.056810483, 0.49693507, 0.1475...",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,Arkeologi,{{Ilmu}}\n[[Berkas:Bulgandry Aboriginal Site.J...,"[[-0.70683825, -0.15112978, 1.1264209, 0.36113...",0.000000,0.004808,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.025237,0.000000,0.0,0.0,0.0,0.0
4,Antropologi,{{Zoologi}}\n'''Antropologi''' adalah ilmu ten...,"[[-0.6152643, -0.25781155, 0.6689638, 0.200405...",0.000000,0.000000,0.000000,0.00000,0.002667,0.002735,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Sitoplasma,'''Sitoplasma''' adalah bagian [[sel (biologi)...,"[[-0.5021529, -0.23019293, 0.25113472, 0.34530...",0.000000,0.000000,0.046385,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.042044,0.0,0.0,0.0,0.0
996,Soto bandung,<sha1>c0bp72uygo4t24toimesri4fd42tp98</s...,"[[-0.0610977, -0.01826957, 0.099067636, 0.0278...",0.000000,0.000000,0.000000,0.00000,0.023740,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
997,Aksara Kanji,<sha1>dr35x91ivwltxqxrz7ty22vnj81ee6t</s...,"[[-0.05445163, 0.008003685, 0.09989598, 0.0192...",0.000000,0.000000,0.000000,0.05377,0.000000,0.000000,0.017491,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
998,Karnaparwa,'''Karnaparwa''' merupakan kitab ke-8 dari ser...,"[[-0.46792656, -0.050959587, 0.27269533, 0.317...",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


In [14]:
# Print model information
print(f"Dimension of word vectors: {model.get_dimension()}")
print(f"Number of words in the model's vocabulary: {len(model.words)}")

# Display a subset of the vocabulary (first 10 words as an example)
print("\nSample vocabulary:")
for word in model.words[:10]:
    print(word)

Dimension of word vectors: 100
Number of words in the model's vocabulary: 33803

Sample vocabulary:
</s>
dan
yang
di
|
=
*
dari
dengan
pada


In [17]:
print(model.get_nearest_neighbors('yurisdiksi'))

[(0.9601615071296692, 'Yurisdiksi'), (0.8058302998542786, 'yuridis'), (0.7680360674858093, 'sanksi'), (0.7655134201049805, 'Sanksi'), (0.7619509100914001, 'substitusi'), (0.7529696822166443, 'keputusan-keputusan'), (0.7419447302818298, 'RRT.'), (0.7334240674972534, 'Kasus-kasus'), (0.7326633334159851, 'hierarki'), (0.7261871099472046, 'Argentina|loc=Pasal')]


In [18]:
# Extract word vectors and words
words = model.words
vectors = [model.get_word_vector(word) for word in words]

# Create a DataFrame to store the word vectors
word_vectors_df = pd.DataFrame(np.array(vectors), index=words)

# Save the DataFrame to a CSV file (optional)
word_vectors_df.to_csv('word_vectors_wikidump.csv')

# Display the DataFrame (optional)
print(word_vectors_df.head())

            0         1         2         3         4         5         6   \
</s> -0.709214  0.332566  0.464057 -0.066941 -0.002832  0.020200  0.269513   
dan   0.204001  0.111476 -0.483633  0.203069  0.426410  0.055451  0.127261   
yang -0.078800 -0.362293 -0.054298  0.056790  0.464049  0.094025  0.239993   
di    0.238683 -0.515318  0.011460  0.379364  0.168259  0.052955  0.451436   
|    -0.709011  0.957120  0.824338 -0.907836  0.546454  0.597995 -0.099724   

            7         8         9   ...        90        91        92  \
</s> -0.203361  0.296272  0.770603  ... -0.301563 -0.237752  0.329383   
dan   0.011987  0.228900  0.001015  ...  0.307391 -0.736611  0.568424   
yang -0.023729 -0.081763 -0.200123  ... -0.128836 -0.466235  0.631626   
di   -0.043969  0.049173  0.249179  ... -0.225816 -0.312563  0.283059   
|    -0.431913  0.110464  1.153236  ...  0.741827 -0.074968  0.264712   

            93        94        95        96        97        98        99  
</s>  0.014575 

In [19]:
word_vectors_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
</s>,-0.709214,0.332566,0.464057,-0.066941,-0.002832,0.020200,0.269513,-0.203361,0.296272,0.770603,...,-0.301563,-0.237752,0.329383,0.014575,0.751032,-0.481075,0.309466,0.328963,-0.432444,-0.128715
dan,0.204001,0.111476,-0.483633,0.203069,0.426410,0.055451,0.127261,0.011987,0.228900,0.001015,...,0.307391,-0.736611,0.568424,0.024860,-0.445762,-0.473743,-0.717672,-0.108403,-0.229697,0.063446
yang,-0.078800,-0.362293,-0.054298,0.056790,0.464049,0.094025,0.239993,-0.023729,-0.081763,-0.200123,...,-0.128836,-0.466235,0.631626,0.050538,-0.082644,-0.000546,0.022589,-0.260003,0.051878,0.126830
di,0.238683,-0.515318,0.011460,0.379364,0.168259,0.052955,0.451436,-0.043969,0.049173,0.249179,...,-0.225816,-0.312563,0.283059,0.239169,-0.244850,-0.413455,-0.849119,-0.290173,-0.391982,-0.081504
|,-0.709011,0.957120,0.824338,-0.907836,0.546454,0.597995,-0.099724,-0.431913,0.110464,1.153236,...,0.741827,-0.074968,0.264712,0.282183,0.212575,-0.212189,-0.379872,-0.319124,-1.253098,0.553355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"magnet]],",-0.179942,-0.235581,0.206347,-0.890774,0.271104,0.412157,0.872846,0.054502,0.432168,-0.332200,...,-0.414157,-0.200626,0.429686,0.000385,0.360149,-0.051527,-0.083353,-0.096879,0.414284,0.928566
contain,-0.250502,0.060435,0.930522,0.630421,0.639804,0.974020,-0.385957,-0.514205,0.164530,0.072785,...,-0.064206,0.477985,-0.003017,0.470483,0.706312,0.194321,0.134242,-0.036298,-0.574534,-0.399778
{{Satuan,-0.497737,0.087022,0.555464,0.230302,0.429306,0.462019,0.438521,0.154743,0.560364,0.348284,...,0.340709,-0.649845,0.324900,-0.624614,0.246956,0.013704,-0.067760,-0.276859,-0.387503,0.685902
{{utama|Transportasi,0.024638,0.137350,0.912255,0.163343,-0.046772,0.282577,0.214261,-0.258150,0.576870,0.511720,...,0.232424,0.070439,0.454727,0.201425,0.314280,-0.193866,0.070669,-0.114877,-0.191022,0.300388
